# Manually creating datasets for testing

In the development version, the process of data cleaning and processing has to be done programmatically on a daily basis. It is, hence, beneficial to construct a few datasets for testing the code.

In [0]:
# LOAD THE REPOSITORY
# if you are working from outside the repository
# this happens if you use colab like me, and start with the notebook
!git clone https://github.com/sandeshbhatjr/energy-prediction.git
!pip install -U --quiet holidays entsoe-py pandas statsmodels

The following cell should extract the german DA price dataset without correcting for DST. This will be called the `raw_df`.

In [6]:
import datetime as dt
import numpy as np
import pandas as pd
from urllib.parse import urljoin

root_path = 'energy-prediction/data/Germany/day-ahead-price/'
df1 = pd.read_csv(urljoin(root_path, 'Day-ahead_prices_201501010000_201701012359_1.csv'), sep=';')
df2 = pd.read_csv(urljoin(root_path, 'Day-ahead_prices_201701010000_201901012359_1.csv'), sep=';')
df3 = pd.read_csv(urljoin(root_path, 'Day-ahead_prices_201901010000_202002062359_1.csv'), sep=';')
df = df1.append(df2[df2['Date'] != 'Jan 1, 2017']).append(df3[df3['Date'] != 'Jan 1, 2019'])

df['Date'] = pd.to_datetime(df['Date'])
df['Time of day'] = pd.to_datetime(df['Time of day']).dt.time
df.replace('-', 'NaN', inplace=True)
countries = list(filter(lambda x: x.endswith('[Euro/MWh]'), sorted(df)))
for country in countries:
  df[country] = df[country].astype('float')
df['timedate'] = pd.to_datetime(df['Date'].astype('str') + ' ' + df['Time of day'].astype('str'))
df.set_index('timedate', drop=True, inplace=True)
df.index.name = None
df.drop(columns=['Date', 'Time of day'], inplace=True)

pre_2018_german_df = \
df[['Germany/Austria/Luxembourg[Euro/MWh]']] \
[:dt.datetime(2018, 9, 30, 22, 0, 0)] \
.copy() \
.rename(columns={'Germany/Austria/Luxembourg[Euro/MWh]': 'Day Ahead Price'})
pre_2018_german_df['Bidding Zone'] = 'DE-AT-LU'

post_2018_german_df = \
df[['Germany/Luxembourg[Euro/MWh]']] \
[dt.datetime(2018, 9, 30, 23, 0, 0):] \
.copy() \
.rename(columns={'Germany/Luxembourg[Euro/MWh]': 'Day Ahead Price'})
post_2018_german_df['Bidding Zone'] = 'DE-LU'

german_df = pre_2018_german_df.append(post_2018_german_df)

german_df.sample(5)

,Day Ahead Price,Bidding Zone
2016-06-26 23:00:00,28.12,DE-AT-LU
2018-02-14 22:00:00,31.37,DE-AT-LU
2016-04-22 19:00:00,30.93,DE-AT-LU
2017-04-07 23:00:00,37.93,DE-AT-LU
2018-11-04 08:00:00,45.48,DE-LU


We save this as a preliminary input for our tests.

In [0]:
german_df.to_hdf('manually_processed_dataframes', key='raw_df')